## Find vertical and horizontal components of the upwelling current in CNT and 3D experiments


In [10]:
#KRM

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.colors as mcolors

from math import *

import scipy.io

import scipy as spy

%matplotlib inline

from netCDF4 import Dataset

import pylab as pl

import os 

import sys

import seaborn as sns

In [11]:
lib_path = os.path.abspath('/ocean/kramosmu/Building_canyon/BuildCanyon/PythonModulesMITgcm') # Add absolute path to my python scripts
sys.path.append(lib_path)

import ReadOutTools_MITgcm as rout 

import ShelfBreakTools_MITgcm as sb


In [12]:
sns.set()
sns.set_style('white')
sns.set_context('talk')

In [32]:
# Functions
def UpwellingVolAvg(Mask,Fld,ztop=30,zbot=37,xups=120,xdowns=240,yshore=267,yopen=227, to=10,tf=19):
    '''Fld : unstaggered field to average
       Mask: C grid mask'''
    
    mask_expand = np.expand_dims(Mask[ztop:zbot,yopen:yshore,xups:xdowns],0)
    mask_expand = mask_expand + np.zeros(Fld[to:tf,ztop:zbot,yopen:yshore,xups:xdowns].shape)

    Fld_small = Fld[to:tf,ztop:zbot,yopen:yshore,xups:xdowns]
    Avg= np.mean(np.ma.masked_array(Fld_small,mask = mask_expand),axis=0)
    
    return np.max(Avg)


In [33]:
# Functions
def UpwellingSpdAvg(Mask,Uvel,Vvel,ztop=30,zbot=37,xups=120,xdowns=240,yshore=267,yopen=227, to=10,tf=19):
    '''Uvel : unstaggered U component
       Vvel : unstaggered V component
       Mask: C grid mask'''
    
    mask_expand = np.expand_dims(Mask[ztop:zbot,yopen:yshore,xups:xdowns],0)
    mask_expand = mask_expand + np.zeros(Uvel[to:tf,ztop:zbot,yopen:yshore,xups:xdowns].shape)

    U_small = Uvel[to:tf,ztop:zbot,yopen:yshore,xups:xdowns]
    V_small = Vvel[to:tf,ztop:zbot,yopen:yshore,xups:xdowns]
    spd = np.sqrt(U_small**2 +V_small**2)
    AvgSpd= np.mean(np.ma.masked_array(spd,mask = mask_expand),axis=0)
    
    return np.max(AvgSpd)


In [37]:
# General input
nx = 360
ny = 360
nz = 90
nt = 19 # t dimension size 


#All experiments in CNT and 3D except no canyon one (run07)
expList = ['/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run02',
           '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run03',
          '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run04',
          '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run09',
          '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run10',
          '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run11',
          '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run12',
          '/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run04',
          '/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run05',
          '/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run06',
          '/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run07']


In [16]:
# The grid is the same for all experiments
CanyonGrid='/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run03/gridGlob.nc'
CanyonGridOut = Dataset(CanyonGrid)

xc = rout.getField(CanyonGrid, 'XC') # x coords tracer cells
yc = rout.getField(CanyonGrid, 'YC') # y coords tracer cells
rc =  CanyonGridOut.variables['RC'] # y coords tracer cells

bathy = rout.getField(CanyonGrid, 'Depth')
hFacC = rout.getField(CanyonGrid, 'HFacC')
MaskC = rout.getMask(CanyonGrid, 'HFacC')
rA = rout.getField(CanyonGrid, 'rA')


### Find average vertical velocity and horizontal velocity within the canyon for each run

In [38]:
Omega = np.zeros(np.shape(expList))
Spd = np.zeros(np.shape(expList))

ii = 0

for exp in expList:
    
    CanyonState= exp +'/stateGlob.nc'
    CanyonStateOut = Dataset(CanyonState)
    z = CanyonStateOut.variables['Z']
    #print(z[37])
    Time = CanyonStateOut.variables['T']
    
    W = rout.getField(CanyonState,'W')
    Ustag = rout.getField(CanyonState,'U')
    Vstag = rout.getField(CanyonState,'V')
    
    U,V = rout.unstagger(Ustag,Vstag)
    print('done with vars')
    Omega[ii] = UpwellingVolAvg(MaskC,W,ztop=30,zbot=37,xups=120,xdowns=240,yshore=267,yopen=227, to=10,tf=19)
    Spd[ii] = UpwellingSpdAvg(MaskC,U,V,ztop=30,zbot=37,xups=120,xdowns=240,yshore=267,yopen=227, to=10,tf=19)
    print(ii,Omega[ii], Spd[ii])
    
    ii = ii+1


done with vars
(0, 0.00045448738253778883, 0.11794462468889025)
done with vars
(1, 0.00047388315821687382, 0.11769391430748834)
done with vars
(2, 0.00048389606591727998, 0.11797001626756456)
done with vars
(3, 0.00045448738253778883, 0.11794462468889025)
done with vars
(4, 0.00047388315821687382, 0.11769391430748834)
done with vars
(5, 0.00050575938075780869, 0.12017046080695258)
done with vars
(6, 0.00066237653502159647, 0.12329374419318305)
done with vars
(7, 0.00053769805365138583, 0.10586559772491455)
done with vars
(8, 0.00047219244556294545, 0.11636047893100315)
done with vars
(9, 0.00054023668376935855, 0.10611693064371745)
done with vars
(10, 0.00047599332821038034, 0.1168063481648763)


In [44]:
import pandas as pd

raw_data = {'run':expList, 'Omega': Omega, 'Ustar': Spd}

df = pd.DataFrame(raw_data, columns = ['day', 'Omega','Ustar'])
    
filename = ('results/metricsDataFrames/OmegaUstarCNTand3D.csv')
df.to_csv(filename)
  
